In [1]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

# 导入模块
from DataCombine_drug_nci import GetData
from ourMethod_gpu import Model
# from compareNumpyMethod import Model
import numpy as np
import time
import random
from torch.backends import cudnn
import tensorly as tl
import torch
import pickle
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy.special import expit

tl.set_backend('pytorch')


class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', times=5, folds=5, negs = 10,**kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.parameters = kwargs
        self.times = times
        self.folds = folds
        self.negs= negs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = self.folds
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        # avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        #score = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))
            since = time.time()
            for k in range(k_folds):
                train_tensor = np.array(self.drug_drug_data.X, copy=True)
                trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                train_tensor[trainpos_index] = 0
                # S1 = np.mat(self.drug_drug_data.S1)
                # S2 = np.mat(self.drug_drug_data.S2)
    
                # train_X = tl.tensor(train_tensor)
                # S1 = tl.tensor(self.drug_drug_data.S1)
                # S2 = tl.tensor(self.drug_drug_data.S2)
    
                train_X = torch.tensor(train_tensor, dtype=torch.float32)
                S1 = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                S2 = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                predict_tensor,M,C,D = self.model()(train_X, S1, S2,
                                              r=self.parameters['r'],
                                              mu=self.parameters['mu'], eta=self.parameters['eta'],
                                              alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                                              lam=self.parameters['lam'],
                                              tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                                              # device=self.parameters['device']
                                              # net=self.parameters['net']
                                              # epoch=self.parameters['epoch'], batch_size=self.parameters['batch_size'],
                                              # hids_size=self.parameters['hids_size'],
                                              # lr=self.parameters['lr'], weight_decay=self.parameters['weight_decay'],
                                              # k=k, label=np.array(self.drug_drug_data.X, copy=True), train_index=train_index,
                                              # Y=np.array(self.drug_drug_data.X, copy=True)
                                              )
    
                print(k,'fold end!')
                fname='NCTF_embeds/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([M,C,D], f)
                
                del M,C,D
                
                #testpos_index 和 posIndex_test是一样的
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = tuple(torch.cat((posIndex_test, negIndex_test), dim=0).numpy().T)
                #print(idxs_test)
    
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                
                ### 获得预测值
                preds = predict_tensor[idxs_test].flatten()
                #print(labels_test.cpu().numpy().shape, preds.shape)
                #metrics_tensor = metrics_tensor + self.get_metrics_1(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_2(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_3(labels_test.cpu().numpy(), preds)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+'NCTF_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([predict_tensor,idxs_test,labels_test.cpu().numpy(),preds], f)


                results = pd.DataFrame({
                    'time': [i] * len(idxs_test[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test[0]),  # 假设这是第 1 次
                    'm1': idxs_test[0],
                    'm2': idxs_test[1],
                    'd': idxs_test[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+'NCTF_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)

                metrics=self.get_metrics_1(labels_test.cpu().numpy(), preds)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics[0])
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j=j+1
            
            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t',i+1,':\t',result)
            #avgmetrics_tensor_10 = avgmetrics_tensor_10 + result
            time_elapsed = time.time() - since
            print(time_elapsed // 60, time_elapsed % 60)

        fname = os.path.join('compareTF', 'NCTF_ddi_'+str(self.negs)+'neg_results_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        print(j)
        #print(df)
        #print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t',results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t',results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score=np.mat(real_score)
        predict_score=np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        #sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]
    
        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1
    
        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN
    
        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T
    
        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])
    
        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])
    
        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)
    
        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]
    
        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

if __name__ == '__main__':
    fix_seed(2024)
    #neg =10
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=11,neg=neg)
    since = time.time()
    #print(drug_drug_data)
    #print(drug_drug_data.shape)
    ### Split
    times=5
    folds=5
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    j=0
    r=57
    mu,eta,alpha,beta,lam=0.5,2,0.125,0.125,0.001
    #for neg in [1,2,4,6,8,10]:
    for neg in [1]:
        signal = 13
        miRNA_num = 87
        disease_num = 55
        folder = '/mnt/sda/liupei/NCTF_new/data/NCI_ddi5cv'
        drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
        experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32', times=times, folds=folds,negs=neg,
                                 r=r, mu=mu, alpha=alpha, eta=eta, beta=beta, lam=lam, tol=1e-4, max_iter=100)
        aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
        df.loc[j] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
        print(f"neg={neg}")
        print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
        j=j+1

    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_1', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_2', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    df.to_csv('NCTF_1negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)


/home/liupei/miniconda3/envs/pyg/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


(87, 87)
(55, 55)
0
1
2
3
4
5
6
7
8
9
(87, 87, 55)
17978.0
(87, 87, 55)
0 fold end!
NCTF_embeds/factors_0_times_0_fold.pkl
pred_score_pkl/NCTF_ddi_0_times_0_foldscores.pkl
1 fold end!
NCTF_embeds/factors_0_times_1_fold.pkl
pred_score_pkl/NCTF_ddi_0_times_1_foldscores.pkl
2 fold end!
NCTF_embeds/factors_0_times_2_fold.pkl
pred_score_pkl/NCTF_ddi_0_times_2_foldscores.pkl
3 fold end!
NCTF_embeds/factors_0_times_3_fold.pkl
pred_score_pkl/NCTF_ddi_0_times_3_foldscores.pkl
4 fold end!
NCTF_embeds/factors_0_times_4_fold.pkl
pred_score_pkl/NCTF_ddi_0_times_4_foldscores.pkl
Times:	 1 :	 [[0.5752 0.8492 0.5419 0.8872 0.5497 0.9338 0.5349]]
5.0 2.4748804569244385
0 fold end!
NCTF_embeds/factors_1_times_0_fold.pkl
pred_score_pkl/NCTF_ddi_1_times_0_foldscores.pkl
1 fold end!
NCTF_embeds/factors_1_times_1_fold.pkl
pred_score_pkl/NCTF_ddi_1_times_1_foldscores.pkl
2 fold end!
NCTF_embeds/factors_1_times_2_fold.pkl
pred_score_pkl/NCTF_ddi_1_times_2_foldscores.pkl
3 fold end!
NCTF_embeds/factors_1_times

In [1]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

# 导入模块
from DataCombine_drug_nci import GetData
#from ourMethod_gpu import Model
from compareNumpyMethod import Model
import numpy as np
import time
import random
from torch.backends import cudnn
import tensorly as tl
import torch
import pickle
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy.special import expit

class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', times=5, folds=5, negs = 10,**kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.parameters = kwargs
        self.times = times
        self.folds = folds
        self.negs= negs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = self.folds
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        # avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        #score = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))
            since = time.time()
            for k in range(k_folds):
                train_tensor = np.array(self.drug_drug_data.X, copy=True)
                trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                train_tensor[trainpos_index] = 0
                S1 = np.mat(self.drug_drug_data.S1)
                S2 = np.mat(self.drug_drug_data.S2)

                predict_tensor,M,C,D = self.model()(train_tensor, S1, S2,
                                              r=self.parameters['r'],
                                              mu=self.parameters['mu'], eta=self.parameters['eta'],
                                              alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                                              lam=self.parameters['lam'],
                                              tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                                              # device=self.parameters['device']
                                              # net=self.parameters['net']
                                              # epoch=self.parameters['epoch'], batch_size=self.parameters['batch_size'],
                                              # hids_size=self.parameters['hids_size'],
                                              # lr=self.parameters['lr'], weight_decay=self.parameters['weight_decay'],
                                              # k=k, label=np.array(self.drug_drug_data.X, copy=True), train_index=train_index,
                                              # Y=np.array(self.drug_drug_data.X, copy=True)
                                              )
    
                print(k,'fold end!')
                fname='CTF_embeds/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([M,C,D], f)
                
                del M,C,D
                
                #testpos_index 和 posIndex_test是一样的
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = tuple(torch.cat((posIndex_test, negIndex_test), dim=0).numpy().T)
                #print(idxs_test)
    
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                
                ### 获得预测值
                preds = predict_tensor[idxs_test].flatten()
                #print(labels_test.cpu().numpy().shape, preds.shape)
                #metrics_tensor = metrics_tensor + self.get_metrics_1(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_2(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_3(labels_test.cpu().numpy(), preds)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+'CTF_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([predict_tensor,idxs_test,labels_test.cpu().numpy(),preds], f)


                results = pd.DataFrame({
                    'time': [i] * len(idxs_test[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test[0]),  # 假设这是第 1 次
                    'm1': idxs_test[0],
                    'm2': idxs_test[1],
                    'd': idxs_test[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+'CTF_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)

                metrics=self.get_metrics_1(labels_test.cpu().numpy(), preds)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics[0])
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j=j+1
            
            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t',i+1,':\t',result)
            #avgmetrics_tensor_10 = avgmetrics_tensor_10 + result
            time_elapsed = time.time() - since
            print(time_elapsed // 60, time_elapsed % 60)

        fname = os.path.join('compareTF', 'CTF_ddi_'+str(self.negs)+'neg_results_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        print(j)
        #print(df)
        #print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t',results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t',results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score=np.mat(real_score)
        predict_score=np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        #sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]
    
        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1
    
        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN
    
        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T
    
        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])
    
        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])
    
        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)
    
        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]
    
        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

if __name__ == '__main__':
    fix_seed(2024)
    #neg =10
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=11,neg=neg)
    since = time.time()
    #print(drug_drug_data)
    #print(drug_drug_data.shape)
    ### Split
    times=5
    folds=5
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    j=0
    r=57
    mu,eta,alpha,beta,lam=0.5,2,0.125,0.125,0.001
    #for neg in [1,2,4,6,8,10]:
    for neg in [1]:
        signal = 23
        miRNA_num = 87
        disease_num = 55
        folder = '/mnt/sda/liupei/NCTF_new/data/NCI_ddi5cv'
        drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
        experiment = Experiments(drug_drug_data, model_name='CTF',  times=times, folds=folds,negs=neg,
                             r=r,  mu=0.5, eta=0.2, alpha=0.5, beta=0.5, lam=0.5, tol=1e-6, max_iter=100)
        aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
        df.loc[j] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
        print(f"neg={neg}")
        print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
        j=j+1

    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_1', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_2', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    df.to_csv('CTF_1negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)


/home/liupei/miniconda3/envs/pyg/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


(87, 87)
(87, 87)
0
1
2
3
4
5
6
7
8
9
(87, 87, 55)
17978.0
(87, 87, 55)
0 fold end!
CTF_embeds/factors_0_times_0_fold.pkl
pred_score_pkl/CTF_ddi_0_times_0_foldscores.pkl
1 fold end!
CTF_embeds/factors_0_times_1_fold.pkl
pred_score_pkl/CTF_ddi_0_times_1_foldscores.pkl
2 fold end!
CTF_embeds/factors_0_times_2_fold.pkl
pred_score_pkl/CTF_ddi_0_times_2_foldscores.pkl
3 fold end!
CTF_embeds/factors_0_times_3_fold.pkl
pred_score_pkl/CTF_ddi_0_times_3_foldscores.pkl
4 fold end!
CTF_embeds/factors_0_times_4_fold.pkl
pred_score_pkl/CTF_ddi_0_times_4_foldscores.pkl
Times:	 1 :	 [[0.556  0.8375 0.5275 0.8856 0.5265 0.9351 0.5294]]
0.0 8.111679315567017
0 fold end!
CTF_embeds/factors_1_times_0_fold.pkl
pred_score_pkl/CTF_ddi_1_times_0_foldscores.pkl
1 fold end!
CTF_embeds/factors_1_times_1_fold.pkl
pred_score_pkl/CTF_ddi_1_times_1_foldscores.pkl
2 fold end!
CTF_embeds/factors_1_times_2_fold.pkl
pred_score_pkl/CTF_ddi_1_times_2_foldscores.pkl
3 fold end!
CTF_embeds/factors_1_times_3_fold.pkl
pred_s

In [2]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

# 导入模块
from DataCombine_drug_nci import GetData
#from ourMethod_gpu import Model
from compareNumpyMethod import Model
import numpy as np
import time
import random
from torch.backends import cudnn
import tensorly as tl
import torch
import pickle
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy.special import expit

class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', times=5, folds=5, negs = 10,**kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.parameters = kwargs
        self.times = times
        self.folds = folds
        self.negs= negs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = self.folds
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        # avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        #score = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))
            since = time.time()
            for k in range(k_folds):
                train_tensor = np.array(self.drug_drug_data.X, copy=True)
                trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                train_tensor[trainpos_index] = 0
                S1 = np.mat(self.drug_drug_data.S1)
                S2 = np.mat(self.drug_drug_data.S2)

                predict_tensor = self.model()(train_tensor, S1, S2,
                                              r=self.parameters['r'],
                                              mu=self.parameters['mu'], eta=self.parameters['eta'],
                                              alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                                              lam=self.parameters['lam'],
                                              tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                                              # device=self.parameters['device']
                                              # net=self.parameters['net']
                                              # epoch=self.parameters['epoch'], batch_size=self.parameters['batch_size'],
                                              # hids_size=self.parameters['hids_size'],
                                              # lr=self.parameters['lr'], weight_decay=self.parameters['weight_decay'],
                                              # k=k, label=np.array(self.drug_drug_data.X, copy=True), train_index=train_index,
                                              # Y=np.array(self.drug_drug_data.X, copy=True)
                                              )
    
                # print(k,'fold end!')
                # fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                # print(fname)
                # with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                #     pickle.dump([M,C,D], f)
                
                # del M,C,D
                
                #testpos_index 和 posIndex_test是一样的
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = tuple(torch.cat((posIndex_test, negIndex_test), dim=0).numpy().T)
                #print(idxs_test)
    
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                
                ### 获得预测值
                preds = predict_tensor[idxs_test].flatten()
                #print(labels_test.cpu().numpy().shape, preds.shape)
                #metrics_tensor = metrics_tensor + self.get_metrics_1(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_2(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_3(labels_test.cpu().numpy(), preds)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+'TDRC_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([predict_tensor,idxs_test,labels_test.cpu().numpy(),preds], f)


                results = pd.DataFrame({
                    'time': [i] * len(idxs_test[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test[0]),  # 假设这是第 1 次
                    'm1': idxs_test[0],
                    'm2': idxs_test[1],
                    'd': idxs_test[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+'TDRC_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)

                metrics=self.get_metrics_1(labels_test.cpu().numpy(), preds)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics[0])
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j=j+1
            
            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t',i+1,':\t',result)
            #avgmetrics_tensor_10 = avgmetrics_tensor_10 + result
            time_elapsed = time.time() - since
            print(time_elapsed // 60, time_elapsed % 60)

        fname = os.path.join('compareTF', 'TDRC_ddi_'+str(self.negs)+'neg_results_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        print(j)
        #print(df)
        #print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t',results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t',results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score=np.mat(real_score)
        predict_score=np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        #sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]
    
        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1
    
        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN
    
        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T
    
        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])
    
        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])
    
        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)
    
        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]
    
        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

if __name__ == '__main__':
    fix_seed(2024)
    #neg =10
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=11,neg=neg)
    since = time.time()
    #print(drug_drug_data)
    #print(drug_drug_data.shape)
    ### Split
    times=5
    folds=5
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    j=0
    r=57
    #mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001 ### 旧的NCTF参数组合 再用
    #mu,eta,alpha,beta,lam=0.5,0.75,0.5,0.125,0.001 ### 新的NCTF参数组合 不用
    #for neg in [1,2,4,6,8,10]:
    for neg in [1]:
        signal = 23
        miRNA_num = 87
        disease_num = 55
        folder = '/mnt/sda/liupei/NCTF_new/data/NCI_ddi5cv'
        drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
        experiment = Experiments(drug_drug_data, model_name='TDRC_mm',  times=times, folds=folds,negs=neg,
                                 r=r, mu=0.125, eta=0.25, alpha=2.0, beta=0.125, lam=0.001,  tol=1e-6, max_iter=100)
        aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
        df.loc[j] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
        print(f"neg={neg}")
        print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
        j=j+1

    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_1', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_2', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    df.to_csv('TDRC_1negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)


(87, 87)
(87, 87)
0
1
2
3
4
5
6
7
8
9
(87, 87, 55)
17978.0
(87, 87, 55)
pred_score_pkl/TDRC_ddi_0_times_0_foldscores.pkl
pred_score_pkl/TDRC_ddi_0_times_1_foldscores.pkl
pred_score_pkl/TDRC_ddi_0_times_2_foldscores.pkl
pred_score_pkl/TDRC_ddi_0_times_3_foldscores.pkl
pred_score_pkl/TDRC_ddi_0_times_4_foldscores.pkl
Times:	 1 :	 [[0.5354 0.8281 0.5162 0.8839 0.511  0.9353 0.5229]]
0.0 22.40919017791748
pred_score_pkl/TDRC_ddi_1_times_0_foldscores.pkl
pred_score_pkl/TDRC_ddi_1_times_1_foldscores.pkl
pred_score_pkl/TDRC_ddi_1_times_2_foldscores.pkl
pred_score_pkl/TDRC_ddi_1_times_3_foldscores.pkl
pred_score_pkl/TDRC_ddi_1_times_4_foldscores.pkl
Times:	 2 :	 [[0.5318 0.8274 0.5125 0.8776 0.5305 0.9255 0.4977]]
0.0 22.597285985946655
pred_score_pkl/TDRC_ddi_2_times_0_foldscores.pkl
pred_score_pkl/TDRC_ddi_2_times_1_foldscores.pkl
pred_score_pkl/TDRC_ddi_2_times_2_foldscores.pkl
pred_score_pkl/TDRC_ddi_2_times_3_foldscores.pkl
pred_score_pkl/TDRC_ddi_2_times_4_foldscores.pkl
Times:	 3 :	 [[0

In [3]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

# 导入模块
from DataCombine_drug_nci import GetData
#from ourMethod_gpu import Model
from compareNumpyMethod import Model
import numpy as np
import time
import random
from torch.backends import cudnn
import tensorly as tl
import torch
import pickle
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy.special import expit

class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', times=5, folds=5, negs = 10,**kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.parameters = kwargs
        self.times = times
        self.folds = folds
        self.negs= negs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = self.folds
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        # avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        #score = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))
            since = time.time()
            for k in range(k_folds):
                train_tensor = np.array(self.drug_drug_data.X, copy=True)
                trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                train_tensor[trainpos_index] = 0
                S1 = np.mat(self.drug_drug_data.S1)
                S2 = np.mat(self.drug_drug_data.S2)

                predict_tensor = self.model()(train_tensor, S1, S2,
                                              r=self.parameters['r'],
                                              mu=self.parameters['mu'], eta=self.parameters['eta'],
                                              alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                                              lam=self.parameters['lam'],
                                              tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                                              # device=self.parameters['device']
                                              # net=self.parameters['net']
                                              # epoch=self.parameters['epoch'], batch_size=self.parameters['batch_size'],
                                              # hids_size=self.parameters['hids_size'],
                                              # lr=self.parameters['lr'], weight_decay=self.parameters['weight_decay'],
                                              # k=k, label=np.array(self.drug_drug_data.X, copy=True), train_index=train_index,
                                              # Y=np.array(self.drug_drug_data.X, copy=True)
                                              )
    
                # print(k,'fold end!')
                # fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                # print(fname)
                # with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                #     pickle.dump([M,C,D], f)
                
                # del M,C,D
                
                #testpos_index 和 posIndex_test是一样的
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = tuple(torch.cat((posIndex_test, negIndex_test), dim=0).numpy().T)
                #print(idxs_test)
    
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                
                ### 获得预测值
                preds = predict_tensor[idxs_test].flatten()
                #print(labels_test.cpu().numpy().shape, preds.shape)
                #metrics_tensor = metrics_tensor + self.get_metrics_1(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_2(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_3(labels_test.cpu().numpy(), preds)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+'CP_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([predict_tensor,idxs_test,labels_test.cpu().numpy(),preds], f)


                results = pd.DataFrame({
                    'time': [i] * len(idxs_test[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test[0]),  # 假设这是第 1 次
                    'm1': idxs_test[0],
                    'm2': idxs_test[1],
                    'd': idxs_test[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+'CP_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)

                metrics=self.get_metrics_1(labels_test.cpu().numpy(), preds)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics[0])
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j=j+1
            
            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t',i+1,':\t',result)
            #avgmetrics_tensor_10 = avgmetrics_tensor_10 + result
            time_elapsed = time.time() - since
            print(time_elapsed // 60, time_elapsed % 60)

        fname = os.path.join('compareTF', 'CP_ddi_'+str(self.negs)+'neg_results_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        print(j)
        #print(df)
        #print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t',results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t',results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score=np.mat(real_score)
        predict_score=np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        #sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]
    
        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1
    
        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN
    
        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T
    
        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])
    
        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])
    
        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)
    
        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]
    
        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

if __name__ == '__main__':
    fix_seed(2024)
    #neg =10
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=11,neg=neg)
    since = time.time()
    #print(drug_drug_data)
    #print(drug_drug_data.shape)
    ### Split
    times=5
    folds=5
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    j=0
    r=57
    #mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001 ### 旧的NCTF参数组合 再用
    #mu,eta,alpha,beta,lam=0.5,0.75,0.5,0.125,0.001 ### 新的NCTF参数组合 不用
    #for neg in [1,2,4,6,8,10]:
    for neg in [1]:
        signal = 23
        miRNA_num = 87
        disease_num = 55
        folder = '/mnt/sda/liupei/NCTF_new/data/NCI_ddi5cv'
        drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
        experiment = Experiments(drug_drug_data, model_name='CP',  times=times, folds=folds,negs=neg,
                         r=r, mu=0.125, eta=0.25, alpha=0.125, beta=0.25, lam=0.001,  tol=1e-6, max_iter=100)
        aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
        df.loc[j] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
        print(f"neg={neg}")
        print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
        j=j+1

    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_1', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_2', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    df.to_csv('CP_1negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)


(87, 87)
(87, 87)
0
1
2
3
4
5
6
7
8
9
(87, 87, 55)
17978.0
(87, 87, 55)
pred_score_pkl/CP_ddi_0_times_0_foldscores.pkl
pred_score_pkl/CP_ddi_0_times_1_foldscores.pkl
pred_score_pkl/CP_ddi_0_times_2_foldscores.pkl
pred_score_pkl/CP_ddi_0_times_3_foldscores.pkl
pred_score_pkl/CP_ddi_0_times_4_foldscores.pkl
Times:	 1 :	 [[0.5619 0.8373 0.5353 0.8892 0.5264 0.9392 0.5452]]
0.0 4.266846179962158
pred_score_pkl/CP_ddi_1_times_0_foldscores.pkl
pred_score_pkl/CP_ddi_1_times_1_foldscores.pkl
pred_score_pkl/CP_ddi_1_times_2_foldscores.pkl
pred_score_pkl/CP_ddi_1_times_3_foldscores.pkl
pred_score_pkl/CP_ddi_1_times_4_foldscores.pkl
Times:	 2 :	 [[0.5607 0.8367 0.5363 0.8886 0.5314 0.9379 0.542 ]]
0.0 4.305094003677368
pred_score_pkl/CP_ddi_2_times_0_foldscores.pkl
pred_score_pkl/CP_ddi_2_times_1_foldscores.pkl
pred_score_pkl/CP_ddi_2_times_2_foldscores.pkl
pred_score_pkl/CP_ddi_2_times_3_foldscores.pkl
pred_score_pkl/CP_ddi_2_times_4_foldscores.pkl
Times:	 3 :	 [[0.5637 0.8393 0.5339 0.8871 0.53

In [4]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

# 导入模块
from DataCombine_drug_nci import GetData
#from ourMethod_gpu import Model
from compareNumpyMethod import Model
import numpy as np
import time
import random
from torch.backends import cudnn
import tensorly as tl
import torch
import pickle
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
from scipy.special import expit

class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', times=5, folds=5, negs = 10,**kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.parameters = kwargs
        self.times = times
        self.folds = folds
        self.negs= negs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = self.folds
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        # avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        #score = pd.DataFrame(columns=['j', 'times', 'folds','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))
            since = time.time()
            for k in range(k_folds):
                train_tensor = np.array(self.drug_drug_data.X, copy=True)
                trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                train_tensor[trainpos_index] = 0
                S1 = np.mat(self.drug_drug_data.S1)
                S2 = np.mat(self.drug_drug_data.S2)

                predict_tensor = self.model()(train_tensor, S1, S2,
                                              r=self.parameters['r'],
                                              mu=self.parameters['mu'], eta=self.parameters['eta'],
                                              alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                                              lam=self.parameters['lam'],
                                              tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                                              # device=self.parameters['device']
                                              # net=self.parameters['net']
                                              # epoch=self.parameters['epoch'], batch_size=self.parameters['batch_size'],
                                              # hids_size=self.parameters['hids_size'],
                                              # lr=self.parameters['lr'], weight_decay=self.parameters['weight_decay'],
                                              # k=k, label=np.array(self.drug_drug_data.X, copy=True), train_index=train_index,
                                              # Y=np.array(self.drug_drug_data.X, copy=True)
                                              )
    
                # print(k,'fold end!')
                # fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                # print(fname)
                # with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                #     pickle.dump([M,C,D], f)
                
                # del M,C,D
                
                #testpos_index 和 posIndex_test是一样的
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = tuple(torch.cat((posIndex_test, negIndex_test), dim=0).numpy().T)
                #print(idxs_test)
    
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                
                ### 获得预测值
                preds = predict_tensor[idxs_test].flatten()
                #print(labels_test.cpu().numpy().shape, preds.shape)
                #metrics_tensor = metrics_tensor + self.get_metrics_1(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_2(labels_test.cpu().numpy(), preds)
                #metrics_tensor = metrics_tensor + self.get_metrics_3(labels_test.cpu().numpy(), preds)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+'TFAI_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([predict_tensor,idxs_test,labels_test.cpu().numpy(),preds], f)


                results = pd.DataFrame({
                    'time': [i] * len(idxs_test[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test[0]),  # 假设这是第 1 次
                    'm1': idxs_test[0],
                    'm2': idxs_test[1],
                    'd': idxs_test[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+'TFAI_ddi_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)

                metrics=self.get_metrics_1(labels_test.cpu().numpy(), preds)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics[0])
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j=j+1
            
            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t',i+1,':\t',result)
            #avgmetrics_tensor_10 = avgmetrics_tensor_10 + result
            time_elapsed = time.time() - since
            print(time_elapsed // 60, time_elapsed % 60)

        fname = os.path.join('compareTF', 'TFAI_ddi_'+str(self.negs)+'neg_results_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        print(j)
        #print(df)
        #print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t',results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t',results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score=np.mat(real_score)
        predict_score=np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        #sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]
    
        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1
    
        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN
    
        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T
    
        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])
    
        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])
    
        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)
    
        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]
    
        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

if __name__ == '__main__':
    fix_seed(2024)
    #neg =10
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=11,neg=neg)
    since = time.time()
    #print(drug_drug_data)
    #print(drug_drug_data.shape)
    ### Split
    times=5
    folds=5
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    j=0
    r=57
    #mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001 ### 旧的NCTF参数组合 再用
    #mu,eta,alpha,beta,lam=0.5,0.75,0.5,0.125,0.001 ### 新的NCTF参数组合 不用
    #for neg in [1,2,4,6,8,10]:
    for neg in [1]:
        signal = 23
        miRNA_num = 87
        disease_num = 55
        folder = '/mnt/sda/liupei/NCTF_new/data/NCI_ddi5cv'
        drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
        experiment = Experiments(drug_drug_data, model_name='TFAI_CP_within_mod',  times=times, folds=folds,negs=neg,
                                 r=r, mu=0.125, eta=0.25, alpha=2.0, beta=0.125, lam=0.001,  tol=1e-6, max_iter=100)
        aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
        df.loc[j] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
        print(f"neg={neg}")
        print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
        j=j+1

    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_1', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    #experiment = Experiments(drug_drug_data, model_name='NCTF_torch_splitSim_2', r=30, mu=0.5, alpha=0.5, eta=0.2, beta=0.5, lam=0.001, tol=1e-6, max_iter=200)
    df.to_csv('TFAI_1negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)


(87, 87)
(87, 87)
0
1
2
3
4
5
6
7
8
9
(87, 87, 55)
17978.0
(87, 87, 55)
pred_score_pkl/TFAI_ddi_0_times_0_foldscores.pkl
pred_score_pkl/TFAI_ddi_0_times_1_foldscores.pkl
pred_score_pkl/TFAI_ddi_0_times_2_foldscores.pkl
pred_score_pkl/TFAI_ddi_0_times_3_foldscores.pkl
pred_score_pkl/TFAI_ddi_0_times_4_foldscores.pkl
Times:	 1 :	 [[0.5664 0.8394 0.5379 0.8892 0.5315 0.9386 0.5446]]
0.0 6.169066429138184
pred_score_pkl/TFAI_ddi_1_times_0_foldscores.pkl
pred_score_pkl/TFAI_ddi_1_times_1_foldscores.pkl
pred_score_pkl/TFAI_ddi_1_times_2_foldscores.pkl
pred_score_pkl/TFAI_ddi_1_times_3_foldscores.pkl
pred_score_pkl/TFAI_ddi_1_times_4_foldscores.pkl
Times:	 2 :	 [[0.5663 0.8386 0.5354 0.888  0.5323 0.9371 0.5393]]
0.0 6.20302677154541
pred_score_pkl/TFAI_ddi_2_times_0_foldscores.pkl
pred_score_pkl/TFAI_ddi_2_times_1_foldscores.pkl
pred_score_pkl/TFAI_ddi_2_times_2_foldscores.pkl
pred_score_pkl/TFAI_ddi_2_times_3_foldscores.pkl
pred_score_pkl/TFAI_ddi_2_times_4_foldscores.pkl
Times:	 3 :	 [[0.5

In [4]:
# import pickle

# # 定义文件名
# fname = 'score/NCTF_hmddv3.2_1_times_1_foldscores.pkl'

# # 读取数据
# with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
#     data = pickle.load(f)

# # 解包数据
# predict_tensor, idxs_test, labels_test, preds = data

# # # 打印数据
# # print("Predict Tensor:", predict_tensor)
# # print("Test Indices:", idxs_test)
# # print("Labels Test:", labels_test)
# # print("Predictions:", preds)

# results = pd.DataFrame({
#     'time': [1] * len(idxs_test[0]),  # 假设这是第 1 折
#     'fold': [1] * len(idxs_test[0]),  # 假设这是第 1 次
#     'm1': idxs_test[0],
#     'm2': idxs_test[1],
#     'm3': idxs_test[2],
#     'true_label': labels_test,
#     'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
# })
# # 保存为 CSV 文件
# csv_file = 'score/NCTF_hmddv3.2_1_times_1_foldscores.csv'
# df.to_csv(csv_file, index=False)


In [3]:
predict_tensor[idxs_test]

array([ 1.6545244e-01,  1.2589991e-01,  1.8950567e-02, ...,
        9.8364753e-06,  3.8205550e-04, -2.1582819e-05], dtype=float32)

In [4]:
predict_tensor[idxs_test].flatten()

array([ 1.6545244e-01,  1.2589991e-01,  1.8950567e-02, ...,
        9.8364753e-06,  3.8205550e-04, -2.1582819e-05], dtype=float32)

In [12]:
results = pd.DataFrame({
    'time': [1] * len(idxs_test[0]),  # 假设这是第 1 折
    'fold': [1] * len(idxs_test[0]),  # 假设这是第 1 次
    'm1': idxs_test[0],
    'm2': idxs_test[1],
    'm3': idxs_test[2],
    'true_label': labels_test,
    'pred_score': preds  # 假设 preds 是一个二维数组，取第二列作为预测概率
})

In [13]:
results

,time,fold,m1,m2,m3,true_label,pred_score
0,1,1,155,154,0,1.0,0.165452
1,1,1,155,155,0,1.0,0.125900
2,1,1,22,22,1,1.0,0.018951
3,1,1,52,53,1,1.0,0.001700
4,1,1,81,82,1,1.0,0.373254
...,...,...,...,...,...,...,...
5867,1,1,165,215,115,0.0,-0.000266
5868,1,1,210,334,195,0.0,0.000410
5869,1,1,44,121,282,0.0,0.000010
5870,1,1,96,249,110,0.0,0.000382
